In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.sql import functions as f

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [12]:
customer = spark.read.option("header", True).csv('classicmodels/csv/customer.csv').createOrReplaceTempView("customer")
order = spark.read.option("header", True).csv('classicmodels/csv/order.csv').createOrReplaceTempView("order")
orderdetail = spark.read.option("header", True).csv('classicmodels/csv/orderdetail.csv').createOrReplaceTempView("orderdetail")
employee = spark.read.option("header", True).csv('classicmodels/csv/employee.csv').createOrReplaceTempView("employee")
product = spark.read.option("header", True).csv('classicmodels/csv/product.csv').createOrReplaceTempView("product")
office = spark.read.option("header", True).csv('classicmodels/csv/office.csv').createOrReplaceTempView("office")

#### Mostra i modellini che costano meno di 75$

In [17]:
view = spark.sql("SELECT MSRP FROM product \
                WHERE MSRP < 75 \
").show(3)

+-----+
| MSRP|
+-----+
| 55.7|
|53.91|
|60.54|
+-----+
only showing top 3 rows



#### Mostra nome, prezzo di acquisto e di vendita dei modellini che costano meno di 75

In [18]:
view = spark.sql("SELECT productName, buyPrice, MSRP FROM product \
                WHERE MSRP < 75 \
").show(3)

+--------------------+--------+-----+
|         productName|buyPrice| MSRP|
+--------------------+--------+-----+
|   1957 Chevy Pickup|    6125| 55.7|
|1936 Mercedes-Ben...|   24.26|53.91|
|  1911 Ford Town Car|    33.3|60.54|
+--------------------+--------+-----+
only showing top 3 rows



#### Mostra tutti i dipendenti di nome Leslie

In [25]:
view = spark.sql("SELECT firstName FROM employee \
                WHERE firstName = 'Leslie' \
").show(3)

+---------+
|firstName|
+---------+
|   Leslie|
|   Leslie|
+---------+



#### Mostra tutti i dipendenti il cui nome finisce per "Arry" e davanti ha una sola lettera

In [30]:
view = spark.sql("SELECT firstName FROM employee \
                WHERE firstName LIKE '_arry' \
").show(3)

+---------+
|firstName|
+---------+
|    Larry|
|    Barry|
+---------+



#### Mostra tutti i prodotti che hanno una scala disivibile per 10 e minore di 100 (es: 1:10, 1:20, 1:30,...)

In [32]:
view = spark.sql("SELECT productCode, productName, productScale FROM product \
                WHERE productScale LIKE '1:_0' \
").show(3)

+-----------+--------------------+------------+
|productCode|         productName|productScale|
+-----------+--------------------+------------+
|   S10_1678|1969 Harley David...|        1:10|
|   S10_1949|1952 Alpine Renau...|        1:10|
|   S10_2016|1996 Moto Guzzi 1...|        1:10|
+-----------+--------------------+------------+
only showing top 3 rows



#### Mostra tutti i dipendenti il cui nome inizia con M e la cui terza lettera è una r

In [35]:
view = spark.sql("SELECT firstName FROM employee \
                WHERE firstName LIKE 'M_r%' \
").show(3)

+---------+
|firstName|
+---------+
|     Mary|
|   Martin|
+---------+



#### MOSTRA I MODELLINI CHE COSTANO MENO DI 75 E CHE ABBIAMO COMPRATO A PIÙ DI 30

In [40]:
view = spark.sql("SELECT productName, buyPrice, MSRP \
                FROM product \
                WHERE MSRP < 75 AND buyPrice > 30 \
").show(5)

+--------------------+--------+-----+
|         productName|buyPrice| MSRP|
+--------------------+--------+-----+
|   1957 Chevy Pickup|    6125| 55.7|
|  1911 Ford Town Car|    33.3|60.54|
|  1934 Ford V8 Coupe|   34.35|62.46|
|    1957 Vespa GS150|   32.95|62.17|
|1957 Ford Thunder...|   34.21|71.27|
+--------------------+--------+-----+
only showing top 5 rows



#### PRENDI TUTTI I DIPENDENTI IL CUI NOME INIZIA CON UNA LETTERA TRA B ED F

In [43]:
view = spark.sql("SELECT firstName FROM employee \
                WHERE firstName LIKE 'B%' OR firstName LIKE 'F%' \
").show(3)

+---------+
|firstName|
+---------+
| Foon Yue|
|    Barry|
+---------+



#### MOSTRA CODICE UFFICIO, CITTÀ E NUMERO DI TELEFONO DEGLI UFFICI IN FRANCIA O AMERICA

In [48]:
view = spark.sql("SELECT officeCode, city, phone FROM office \
                WHERE country = 'France' OR country = 'USA' \
").show(3)

+----------+-------------+---------------+
|officeCode|         city|          phone|
+----------+-------------+---------------+
|         1|San Francisco|+1 650 219 4782|
|         2|       Boston|+1 215 837 0825|
|         3|          NYC|+1 212 555 3000|
+----------+-------------+---------------+
only showing top 3 rows



#### MOSTRARE I MODELLINI DEL TIPO "PLANES","SHIPS" O "CLASSIC CARS"

In [51]:
view = spark.sql("SELECT productCode, productName, productLine FROM product \
                WHERE productLine = 'Planes' OR productLine = 'Ships' or productLine = 'Classic Cars' \
").show(3)

+-----------+--------------------+------------+
|productCode|         productName| productLine|
+-----------+--------------------+------------+
|   S10_1949|1952 Alpine Renau...|Classic Cars|
|   S10_4757| 1972 Alfa Romeo GTA|Classic Cars|
|   S10_4962|1962 LanciaA Delt...|Classic Cars|
+-----------+--------------------+------------+
only showing top 3 rows



#### MOSTRA GLI ORDINI NON SPEDITI

In [71]:
view = spark.sql("SELECT * FROM order \
                WHERE shippedDate IS NULL \
").show(5)

+-----------+---------+------------+-----------+------+--------+--------------+
|orderNumber|orderDate|requiredDate|shippedDate|status|comments|customerNumber|
+-----------+---------+------------+-----------+------+--------+--------------+
+-----------+---------+------------+-----------+------+--------+--------------+



#### MOSTRA I PREZZI DI VENDITA SENZA L’IVA (PREZZO / 1.22)

In [79]:
view = spark.sql("SELECT productName, (MSRP / 1.22) as WITHOUT_IVA \
                FROM product \
").show(3)

+--------------------+------------------+
|         productName|       WITHOUT_IVA|
+--------------------+------------------+
|1969 Harley David...| 78.44262295081968|
|1952 Alpine Renau...|175.65573770491804|
|1996 Moto Guzzi 1...| 97.49180327868852|
+--------------------+------------------+
only showing top 3 rows



#### MOSTRA I PRODOTTI CON UN MARGINE (PREZZO - PREZZO ACQUISTO) SUPERIORE A 50

In [81]:
view = spark.sql("SELECT productName, MSRP, buyPrice, (MSRP - buyPrice) as MARGINE \
                FROM product \
").show(3)

+--------------------+------+--------+------------------+
|         productName|  MSRP|buyPrice|           MARGINE|
+--------------------+------+--------+------------------+
|1969 Harley David...|  95.7|   48.81|             46.89|
|1952 Alpine Renau...| 214.3|   98.58|115.72000000000001|
|1996 Moto Guzzi 1...|118.94|   68.99|             49.95|
+--------------------+------+--------+------------------+
only showing top 3 rows



#### MOSTRA I PRODOTTI CON NOMI DI ALMENO 15 CARATTERI.

In [86]:
view = spark.sql("SELECT productName FROM product \
                WHERE length(productName) > 15 \
").show(3)

+--------------------+
|         productName|
+--------------------+
|1969 Harley David...|
|1952 Alpine Renau...|
|1996 Moto Guzzi 1...|
+--------------------+
only showing top 3 rows



#### MOSTRA I PRODOTTI ORINATI NEL MESE DI GENNAIO

In [90]:
view = spark.sql("SELECT orderNumber, orderDate, customerNumber \
                FROM order \
                WHERE month(orderDate) = 1 \
").show(3)

+-----------+----------+--------------+
|orderNumber| orderDate|customerNumber|
+-----------+----------+--------------+
|      10100|2003-01-06|           363|
|      10101|2003-01-09|           128|
|      10102|2003-01-10|           181|
+-----------+----------+--------------+
only showing top 3 rows



#### MOSTRA I MODELLINI ORDINANDOLI PER PREZZO DI VENDITA CRESCENTE

In [98]:
view = spark.sql("SELECT productName, MSRP FROM product \
                ORDER BY MSRP \
    ").show(3)

+--------------------+---------------+
|         productName|           MSRP|
+--------------------+---------------+
|The USS Constitut...| copper railing|
|         The Titanic|         100.17|
|Collectable Woode...|         100.84|
+--------------------+---------------+
only showing top 3 rows



#### MOSTRA I CLIENTI ORDINANDOLI PER PAESE CRESCENTE E CREDITO MASSIMO DECRESCENTE

In [100]:
view = spark.sql("SELECT customerName, customerNumber, country, creditLimit \
                FROM customer \
                ORDER BY country, creditLimit DESC \
").show(3)

+--------------------+--------------+---------+-----------+
|        customerName|customerNumber|  country|creditLimit|
+--------------------+--------------+---------+-----------+
|  Kelly\'s Gift Shop|           496|   110000|       null|
|Souveniers And Th...|           282|Australia|      93300|
|Australian Collec...|           471|Australia|      60300|
+--------------------+--------------+---------+-----------+
only showing top 3 rows



#### MOSTRA I PRODOTTI VENDUTI A MENO DI 100€, METTENDO IN CIMA QUELLI CON IL MARGINE PIÙ ALTO

In [103]:
view = spark.sql("SELECT productCode, productName, MSRP \
                FROM product \
                WHERE MSRP < 100 \
                ORDER BY MSRP DESC\
").show(3)

+-----------+--------------------+-----+
|productCode|         productName| MSRP|
+-----------+--------------------+-----+
|   S32_1374|    1997 BMW F650 ST|99.89|
|  S700_2466|America West Airl...|99.72|
|  S700_3962|      The Queen Mary|99.31|
+-----------+--------------------+-----+
only showing top 3 rows



#### MOSTRA PER OGNI CLIENTE IL NOME DEL VENDITORE ASSOCIATO

In [104]:
view = spark.sql("SELECT firstName, lastName, customerName \
                FROM customer \
                JOIN employee \
                ON employeeNumber = salesRepEmployeeNumber \
").show(3)

+---------+---------+--------------------+
|firstName| lastName|        customerName|
+---------+---------+--------------------+
|   Gerard|Hernandez|   Atelier graphique|
|   Leslie| Thompson|  Signal Gift Stores|
|     Andy|   Fixter|Australian Collec...|
+---------+---------+--------------------+
only showing top 3 rows



#### MOSTRA TUTTI GLI IMPIEGATI E LA CITTÀ IN CUI SI TROVA L’UFFICIO CUI AFFERISCONO

In [109]:
view = spark.sql("SELECT lastName, firstName, employeeNumber, city \
                FROM employee \
                NATURAL JOIN office \
").show(3)

+---------+---------+--------------+-------------+
| lastName|firstName|employeeNumber|         city|
+---------+---------+--------------+-------------+
|   Murphy|    Diane|          1002|San Francisco|
|Patterson|     Mary|          1056|San Francisco|
| Firrelli|     Jeff|          1076|San Francisco|
+---------+---------+--------------+-------------+
only showing top 3 rows



#### MOSTRA TUTTI I CLIENTI; SE IL CLIENTE HA UN VENDITORE ASSOCIATO, MOSTRANE I DATI

In [119]:
view = spark.sql("SELECT customerName, lastName \
                FROM customer \
                LEFT OUTER JOIN employee \
                ON employeeNumber = salesRepEmployeeNumber \
").show(3)

+--------------------+---------+
|        customerName| lastName|
+--------------------+---------+
|   Atelier graphique|Hernandez|
|  Signal Gift Stores| Thompson|
|Australian Collec...|   Fixter|
+--------------------+---------+
only showing top 3 rows



#### MOSTRA TUTTI I CLIENTI ED I RELATIVI ORDINI, INCLUSI I CLIENTI CHE NON HANNO FATTO ORDINI

In [124]:
view = spark.sql("SELECT c.customerNumber, c.customerName, o.orderNumber \
                FROM customer c\
                RIGHT OUTER JOIN order o\
                on o.customerNumber = c.customerNumber \
").show(3)

+--------------+--------------------+-----------+
|customerNumber|        customerName|orderNumber|
+--------------+--------------------+-----------+
|           363|Online Diecast Cr...|      10100|
|           128|Blauer See Auto, Co.|      10101|
|           181|     Vitachrome Inc.|      10102|
+--------------+--------------------+-----------+
only showing top 3 rows



#### MOSTRA TUTTI I CLIENTI, IL NOME DELL’IMPIEGATO ASSOCIATO ED IL NUMERO DI TELEFONO DELL’UFFICIO

In [132]:
view = spark.sql("SELECT customerName, concat(lastName, ' ', firstName) as employeeName, o.phone \
                FROM customer c\
                LEFT JOIN employee e\
                ON c.salesRepEmployeeNumber = e.employeeNumber \
                LEFT JOIN office o\
                USING(officeCode) \
    ").show(3)

+--------------------+----------------+---------------+
|        customerName|    employeeName|          phone|
+--------------------+----------------+---------------+
|   Atelier graphique|Hernandez Gerard|+33 14 723 4404|
|  Signal Gift Stores| Thompson Leslie|+1 650 219 4782|
|Australian Collec...|     Fixter Andy|+61 2 9264 2451|
+--------------------+----------------+---------------+
only showing top 3 rows



#### STAMPARE OGNI RIGA DELL’ORDINE, INDICANDO IL NOME DEL CLIENTE, NUMERO D’ORDINE ED IL NOME DEL PRODOTTO ORDINATO

In [145]:
view = spark.sql("SELECT customerName, orderNumber, productName \
                FROM order o\
                JOIN customer c \
                USING(customerNumber) \
                JOIN orderdetail od \
                USING(orderNumber) \
                JOIN product \
                USING(productCode) \
                ORDER BY o.orderNumber ASC, od.orderLineNumber \
").show(3)

+--------------------+-----------+--------------------+
|        customerName|orderNumber|         productName|
+--------------------+-----------+--------------------+
|Online Diecast Cr...|      10100|1936 Mercedes Ben...|
|Online Diecast Cr...|      10100|  1911 Ford Town Car|
|Online Diecast Cr...|      10100|1917 Grand Tourin...|
+--------------------+-----------+--------------------+
only showing top 3 rows



#### MOSTRA TUTTI I DIPENDENTI ED IL NOME DEL LORO CAPO

In [150]:
view = spark.sql("SELECT concat(tab1.lastName, ' ', tab1.firstName) as employeeName, tab2.reportsTo \
                FROM employee tab1\
                LEFT JOIN employee tab2 \
                ON tab1.employeeNumber = tab2.reportsTo \
    ").show(3)

+--------------+---------+
|  employeeName|reportsTo|
+--------------+---------+
|  Murphy Diane|     1002|
|  Murphy Diane|     1002|
|Patterson Mary|     1056|
+--------------+---------+
only showing top 3 rows



#### MOSTRA TUTTE LE COPPIE DI CLIENTI CHE ABITANO NELLA STESSA CITTÀ

In [159]:
view = spark.sql("SELECT tab1.customerName, tab2.customerName, tab1.city \
                FROM customer tab1\
                INNER JOIN customer tab2 \
                ON tab1.city = tab2.city AND tab1.customerName <> tab2.customerName\
    ").show(3)

+--------------------+--------------------+---------+
|        customerName|        customerName|     city|
+--------------------+--------------------+---------+
|   Atelier graphique|   La Rochelle Gifts|   Nantes|
|   La Rochelle Gifts|   Atelier graphique|   Nantes|
|Blauer See Auto, Co.|Messner Shopping ...|Frankfurt|
+--------------------+--------------------+---------+
only showing top 3 rows



#### MOSTRA TUTTI GLI STATI DEGLI ORDINI ESISTENTI

In [163]:
view = spark.sql("SELECT status FROM order GROUP BY status").show(3)

+---------+
|   status|
+---------+
|  Shipped|
|  On Hold|
|Cancelled|
+---------+
only showing top 3 rows



#### MOSTRA TUTTI GLI STATI DEGLI ORDINI FATTI PRIMA DEL 31/12/2003

In [171]:
view = spark.sql("SELECT status \
                FROM order \
                WHERE orderDate < '2003-12-31' \
                GROUP BY status \
    ").show(3)

+---------+
|   status|
+---------+
|  Shipped|
|Cancelled|
| Resolved|
+---------+



#### QUANTI CAPI CI SONO IN AZIENDA?

In [184]:
view = spark.sql("SELECT COUNT(DISTINCT reportsTo) as boss \
                FROM employee \
    ").show(3)

+----+
|boss|
+----+
|   7|
+----+



#### MOSTRA GLI STATI DEGLI ORDINI E QUANTI ORDINI SI TROVANO IN CIASCUNO STATO

In [185]:
view = spark.sql("SELECT status, count(orderNumber) \
                FROM order \
                GROUP BY status \
    ").show(3)

+---------+------------------+
|   status|count(orderNumber)|
+---------+------------------+
|  Shipped|               303|
|  On Hold|                 4|
|Cancelled|                 6|
+---------+------------------+
only showing top 3 rows



#### MOSTRARE QUANTI ORDINI HO SPEDITO NEI VARI MESI (UNA RIGA PER MESE ED ANNO)

In [197]:
view = spark.sql("SELECT year(shippedDate), month(shippedDate), COUNT(*) \
                FROM order \
                WHERE shippedDate IS NOT NULL \
                GROUP BY year(shippedDate), month(shippedDate)\
    ").show(3)

+-----------------+------------------+--------+
|year(shippedDate)|month(shippedDate)|count(1)|
+-----------------+------------------+--------+
|             2005|                 5|       8|
|             2004|                 6|       8|
|             2003|                 2|       5|
+-----------------+------------------+--------+
only showing top 3 rows



#### MOSTRARE I CLIENTI CHE NON HANNO FATTO ORDINI

In [198]:
view = spark.sql("SELECT customerName FROM customer \
                WHERE customerNumber NOT IN \
                    (SELECT DISTINCT customerNumber FROM order) \
").show(3)

+--------------------+
|        customerName|
+--------------------+
|  Havel & Zbyszek Co|
|American Souvenir...|
|   Porto Imports Co.|
+--------------------+
only showing top 3 rows



#### MOSTRARE IL NUMERO MASSIMO, MINIMO E MEDIO DI PEZZI INSERITI NEGLI ORDINI

In [204]:
view = spark.sql("SELECT DISTINCT orderNumber, MIN(quantityOrdered), MAX(quantityOrdered), AVG(quantityOrdered) \
                FROM orderdetail \
                GROUP BY orderNumber \
                ").show(3)

+-----------+--------------------+--------------------+--------------------+
|orderNumber|min(quantityOrdered)|max(quantityOrdered)|avg(quantityOrdered)|
+-----------+--------------------+--------------------+--------------------+
|      10100|                  22|                  50|               37.75|
|      10101|                  25|                  46|                35.5|
|      10102|                  39|                  41|                40.0|
+-----------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [207]:
view = spark.sql("SELECT max(items), min(items), avg(items) \
                FROM (SELECT orderNumber, SUM(quantityOrdered) as items FROM orderdetail  \
                    GROUP BY orderNumber) as lineitems \
    ").show(5)

+----------+----------+-----------------+
|max(items)|min(items)|       avg(items)|
+----------+----------+-----------------+
|     717.0|      15.0|323.6687116564417|
+----------+----------+-----------------+



#### MOSTRARE I PRODOTTI IL CUI PREZZO DI ACQUISTO E' SUPERIORE ALLA MEDIA DELLA LINEA CUI AFFERISCONO

In [223]:
view = spark.sql("SELECT productName, buyPrice \
                FROM product AS p\
                WHERE buyPrice > (SELECT AVG(buyPrice) \
                                    FROM product \
                                    WHERE productLine = p.productLine) \
").show(3)

+--------------------+--------+
|         productName|buyPrice|
+--------------------+--------+
|1996 Moto Guzzi 1...|   68.99|
|2003 Harley-David...|   91.02|
|    2002 Suzuki XREO|   66.27|
+--------------------+--------+
only showing top 3 rows



In [214]:
view = spark.sql("SELECT avg(buyPrice), productLine \
                FROM product \
                GROUP BY productLine \
    ").show(5)

+------------------+----------------+
|     avg(buyPrice)|     productLine|
+------------------+----------------+
| 50.68538461538461|     Motorcycles|
|          46.06625|    Vintage Cars|
|48.637499999999996|           Ships|
| 608.0836363636363|Trucks and Buses|
|173.01216216216213|    Classic Cars|
+------------------+----------------+
only showing top 5 rows

